Analyse LP performance on Ekubo ETH/USDC pool by user address

In [1]:
# Standard Library Imports
import math

# Third-Party Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Local Imports
from src.data.ekubo_data import EkuboData  # Importing the EkuboData class
from src.utils.ekubo_math import get_token_order, get_pool_fee_touse, get_tick_spacing_touse

In [5]:
# Tokens in the LP
address1 = '0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8'  # USDC
address2 = '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7'  # ETH

token0_address, token1_address = get_token_order(address1, address2)
print(f"Token0 Address: {token0_address}, Token1 Address: {token1_address}")

# existing values from the website 
pool_fee = 0.05  # in %
tick_spacing = 0.1  # in %

# fee and tick to use
pool_fee_touse = get_pool_fee_touse(pool_fee)
tick_spacing_touse = get_tick_spacing_touse(tick_spacing)

print(f"Pool Fee to use: {pool_fee_touse}")
print(f"Tick Spacing to use: {tick_spacing_touse}")

Token0 Address: 0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7, Token1 Address: 0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8
Pool Fee to use: 170141183460469235273462165868118016
Tick Spacing to use: 1000


In [6]:
# Initialize the data loader with the required parameters
ekubo_data = EkuboData(
    client_url="https://starknet-mainnet.public.blastapi.io", 
    contract_address="0x00000005dd3d2f4429af886cd1a3b08289dbcea99a294197e9eb43b0e0325b4b", # Ekubo
    token0_address="0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7",  # ETH Address
    token1_address="0x53c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8",  # USDC Address
    pool_fee=pool_fee_touse,
    tick_spacing=tick_spacing_touse
)

In [4]:
# Load events from files
events_response_positions_updated = ekubo_data.load_events_from_file('data_downloaded/events_response_positions_updated.pkl')
combined_events_response_swap = ekubo_data.load_events_from_file('data_downloaded/combined_events_response_swap.pkl')
# events_response_positions_updated = events_response_positions_updated.events
print(events_response_positions_updated.events[0])
print(combined_events_response_swap[0])

EmittedEvent(from_address=158098919692956613592021320609952044916245725306097615271255138786123, keys=[1653201459205670538619725765890364513302126541856407219635220342091629827182], data=[1301608360075471435988652873011016112527098238822505666695223302941012672615, 2087021424722619777119509474943472645767659996348769578120564519014510906823, 2368576823837625528275935341135881659748932889268308403712618244410713532584, 0, 1001, 0, 634487, 19631612, 1, 19630611, 1, 4025579270426132218, 1, 0, 0, 109999999866, 1], transaction_hash=3380792800644086123602274049313571131378918353183444763251013603515006644394, block_hash=1007256769720868385100059786946035728971940593564070340067263501679755643489, block_number=655177)
EmittedEvent(from_address=158098919692956613592021320609952044916245725306097615271255138786123, keys=[606811659813332433876493709172014493172264375712319669538606486437773289584], data=[1878172929613570324457239593384623395652998589633516919063222269774660188943, 20870214247226

In [12]:
# Process the events to filter relevant data and create DataFrames
df_liquidity, df_swap = ekubo_data.process_events(events_response_positions_updated.events, combined_events_response_swap)

# Display the first few rows of the liquidity and swap DataFrames
print("Liquidity DataFrame:")
display(df_liquidity.head())
print(len(df_liquidity))
print("Swap DataFrame:")
display(df_swap.head())
print(len(df_swap))


Liquidity DataFrame:


,block_number,transaction_hash,delta_liquidity,tick_lower,tick_upper
0,655189,1870003463488324013557109485794406691956424922...,-4098442033727333,-19560000,-19485000
1,655191,7496014676659025143298985231117091524661156471...,217283698718749,-19659000,-19529000
2,655254,2531281685448483276887619456573047919358036353...,-1930452388810913,-19595000,-19563000
3,655257,2796112001729733432731889769034526094494676095...,-425398360136812,-19649000,-19521000
4,655266,2524055520502035698904493616099606583342588548...,-302553323753,-88722000,88722000


531
Swap DataFrame:


,block_number,transaction_hash,price,tick_id,liquidity_after,amount0,amount1
0,655177,5379951863121538683099002174430011598502365039...,3.073607e-09,-19600424,114379534964441556,737888330085366722,-2267658441
1,655177,5379951863121538683099002174430011598502365039...,3.073373e-09,-19600500,114379534964441556,78955167492407893,-242546697
2,655177,5379951863121538683099002174430011598502365039...,3.073318e-09,-19600518,114379534964441556,18531231477828963,-56924415
3,655177,4855896733655157058425204165595320457715756741...,3.076636e-09,-19599439,113058259809150450,-1105851471796272212,3402163387
4,655177,4106524299860971263060468316553724510380052057...,3.076666e-09,-19599429,113058259809150450,-10362418888953184,31897500


277095
